In [33]:
PROJECT = "train-to-cloud-city-4"
BASE = "http://localhost:8000/"

In [34]:
import requests
import json
from urllib.parse import urljoin

from google.cloud import firestore
db = firestore.Client(project=PROJECT)


In [35]:
res = requests.get(urljoin(BASE,"service/"))
# print(res.text)

service_data = json.loads(res.text)
print(service_data["cloud-run"])

with open("../static/category_map.json", encoding="utf-8") as f:
    category_map = json.loads(f.read())

with open("../static/category_list.json", encoding="utf-8") as f:
    category_list = json.loads(f.read())

print(category_map)
#category_map = {key.replace(r"/", r" & "): value for key, value in category_map.items()}
print(category_map)
print(category_list)


{'slug': 'cloud-run', 'name': 'Cloud Run', 'description': 'Serverless for containerized applications', 'doc_url': 'https://cloud.google.com/run/docs'}
{'Compute': ['cloud-functions', 'app-engine', 'cloud-run', 'cloud-run-for-anthos', 'gke', 'compute-engine', 'bare-metal-solution', 'preemptible-vms', 'sole-tenant-nodes', 'vmware-engine'], 'Storage': ['cloud-filestore', 'cloud-storage', 'local-ssd', 'persistent-disk'], 'Database': ['alloydb', 'cloud-bigtable', 'cloud-firestore', 'cloud-memorystore', 'cloud-sql', 'cloud-sql-insights', 'cloud-spanner'], 'Data Analytics': ['bigquery', 'bigquery-bi-engine', 'bigquery-data-transfer-service', 'bigquery-dts', 'bigquery-gis', 'bigquery-ml', 'cloud-composer', 'cloud-data-fusion', 'connected-sheets', 'data-catalog', 'data-studio', 'dataflow', 'dataplex', 'dataprep-by-trifacta', 'dataproc', 'datastream', 'looker', 'public-datasets'], 'AI/ML': ['automl', 'cloud-tpu', 'cloud-translation', 'cloud-vision', 'contact-center-ai', 'deep-learning-containers

In [36]:

services_ref = db.collection("services")

q = services_ref.count()
#q??
print(q.get())

docs = services_ref.stream()

for i, doc in enumerate(docs):
    print(f"{doc.id} => {doc.to_dict()}")
    if i > 5: break

[[<Aggregation alias=field_1, value=248, readtime=2024-04-05 21:52:33.673878+00:00>]]
access-context-manager => {'slug': 'access-context-manager', 'description': 'Fine-grained, attribute based access-control', 'doc_url': 'https://cloud.google.com/access-context-manager/docs', 'name': 'Access Context Manager'}
access-transparency => {'slug': 'access-transparency', 'description': 'Audit cloud provider access', 'doc_url': 'https://cloud.google.com/cloud-provider-access-management/access-transparency/docs/overview', 'name': 'Access Transparency'}
admin-sdk => {'slug': 'admin-sdk', 'description': 'Manage Google Workspace resources', 'doc_url': 'https://developers.google.com/apps-script/advanced/admin-sdk-directory', 'name': 'Admin SDK'}
alloydb => {'slug': 'alloydb', 'description': 'Scalable & performant PostgreSQL - compatible DB', 'doc_url': 'https://cloud.google.com/alloydb/docs', 'name': 'AlloyDB'}
amp-for-email => {'slug': 'amp-for-email', 'description': 'Dynamic interactive email', 'd

In [37]:
# insert/update service data
for key, data in service_data.items():
    doc_ref = services_ref.document(key)
    doc_ref.set(data)

In [38]:
# add patterns

res = requests.get(urljoin(BASE,"pattern/"))
# print(res.text)
pattern_data = json.loads(res.text)

for pattern in pattern_data.values():
    # print(pattern)
    for checkpoint in pattern["checkpoints"]:
        for service in checkpoint["satisfying_services"]:
            if not ((service in service_data) or (service in {"*"})):
                print (f"'{service}' not found. Referenced in checkpoint: {checkpoint}")

patterns_ref = db.collection("patterns")
for data in pattern_data.values():
    doc_ref = patterns_ref.document(data["slug"])
    doc_ref.set(data)


In [39]:

category_ref = db.collection("services_by_category")

q = category_ref.count()
#q??
print(q.get())

docs = category_ref.stream()

for i, doc in enumerate(docs):
    print(f"{doc.id} => {doc.to_dict()}")
    if i > 5: break

[[<Aggregation alias=field_1, value=8, readtime=2024-04-05 21:52:44.469781+00:00>]]
application-integration => {'services': [{'slug': 'cloud-scheduler', 'description': 'Managed cron job service', 'doc_url': 'https://cloud.google.com/scheduler/docs/', 'name': 'Cloud Scheduler'}, {'slug': 'cloud-tasks', 'description': 'Asynchronous task execution', 'doc_url': 'https://cloud.google.com/tasks/docs/', 'name': 'Cloud Tasks'}, {'slug': 'eventarc', 'description': 'Event-driven Cloud Run services', 'doc_url': 'https://cloud.google.com/eventarc/docs/', 'name': 'Eventarc'}, {'slug': 'pubsub', 'description': 'Global real-time messaging', 'doc_url': 'https://cloud.google.com/pubsub/docs/', 'name': 'Pub/Sub'}, {'slug': 'workflows', 'description': 'HTTP services orchestration', 'doc_url': 'https://cloud.google.com/workflows/docs/', 'name': 'Workflows'}], 'name': 'Application Integration'}
compute => {'services': [{'slug': 'cloud-functions', 'description': 'Event-driven serverless functions', 'doc_url

In [40]:
 # map category data

train_services = set([
"bigquery", "artifact-registry", "kubernetes", "eventarc", "compute-engine", 
"cloud-functions", "sql", "cloud-build","workflows", "cloud-run", 
"secret-manager", "cloud-firestore", "cloud-storage", "cloud-tasks", 
"cloud-scheduler", "pubsub", "iam", "vpc-network", "cloud-deploy", "cloud-memorystore"])
print(f"train_services: {train_services}")

pattern_services = set()
for pattern in pattern_data.values():
    # print(pattern)
    for checkpoint in pattern["checkpoints"]:
        for service in checkpoint["satisfying_services"]:
            # print(service)
            pattern_services.add(service)
print(f"pattern_services: {pattern_services}")
print(f"missing: {train_services.difference(pattern_services)}")

union_services = train_services.union(pattern_services)
categories = {}
for cat, services in category_map.items():
    cat_slug = cat.lower()
    cat_slug = cat_slug.replace(" ", "-")
    cat_slug = cat_slug.replace("/", "-")
    #print({"services": [service_data[slug] for slug in services]})
    categories[cat_slug] = {"services": [service_data[slug] for slug in services if slug in union_services]}
    categories[cat_slug]["name"] = cat
    #print(cat, services)

print(categories["compute"])
print(len(categories))

to_delete = []
for cat, data in categories.items():
    if len(data["services"]) == 0:
        to_delete.append(cat)
for key in to_delete:
    del(categories[key])
print(len(categories))


train_services: {'cloud-functions', 'artifact-registry', 'workflows', 'eventarc', 'secret-manager', 'cloud-run', 'vpc-network', 'compute-engine', 'bigquery', 'cloud-scheduler', 'cloud-deploy', 'cloud-tasks', 'cloud-memorystore', 'cloud-build', 'pubsub', 'cloud-storage', 'sql', 'iam', 'cloud-firestore', 'kubernetes'}
pattern_services: {'cloud-run', 'gke', 'cloud-bigtable', 'cloud-functions', 'alloydb', 'cloud-firestore', 'cloud-deploy', 'cloud-sql', 'artifact-registry', 'cloud-memorystore', 'cloud-spanner', 'app-engine', 'compute-engine', 'cloud-build', 'cloud-storage'}
missing: {'sql', 'bigquery', 'vpc-network', 'iam', 'cloud-scheduler', 'cloud-tasks', 'secret-manager', 'workflows', 'eventarc', 'kubernetes', 'pubsub'}
{'services': [{'slug': 'cloud-functions', 'name': 'Cloud Functions', 'description': 'Event-driven serverless functions', 'doc_url': 'https://cloud.google.com/functions/docs/'}, {'slug': 'app-engine', 'name': 'App Engine', 'description': 'Managed app platform', 'doc_url': 

In [41]:
def delete_collection(coll_ref, batch_size):
    if batch_size == 0:
        return

    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

delete_collection(category_ref, 50)        
# insert/update service_by_category data
for key, data in categories.items():
    doc_ref = category_ref.document(key)
    #print(doc_ref)
    #print(data)
    doc_ref.set(data)

Deleting doc application-integration => {'services': [{'slug': 'cloud-scheduler', 'description': 'Managed cron job service', 'doc_url': 'https://cloud.google.com/scheduler/docs/', 'name': 'Cloud Scheduler'}, {'slug': 'cloud-tasks', 'description': 'Asynchronous task execution', 'doc_url': 'https://cloud.google.com/tasks/docs/', 'name': 'Cloud Tasks'}, {'slug': 'eventarc', 'description': 'Event-driven Cloud Run services', 'doc_url': 'https://cloud.google.com/eventarc/docs/', 'name': 'Eventarc'}, {'slug': 'pubsub', 'description': 'Global real-time messaging', 'doc_url': 'https://cloud.google.com/pubsub/docs/', 'name': 'Pub/Sub'}, {'slug': 'workflows', 'description': 'HTTP services orchestration', 'doc_url': 'https://cloud.google.com/workflows/docs/', 'name': 'Workflows'}], 'name': 'Application Integration'}
Deleting doc compute => {'services': [{'slug': 'cloud-functions', 'description': 'Event-driven serverless functions', 'doc_url': 'https://cloud.google.com/functions/docs/', 'name': 'Cl

In [42]:
# "global" collection now init via `init_global.py` script


In [43]:
docs = services_ref.stream()
for i, doc in enumerate(docs):
    if i >= 5: break
    print(f"{doc.id} => {doc.to_dict()}")
print("----")
    
docs = patterns_ref.stream()
for i, doc in enumerate(docs):
    if i >= 5: break
    print(f"{doc.id} => {doc.to_dict()}")



access-context-manager => {'slug': 'access-context-manager', 'description': 'Fine-grained, attribute based access-control', 'doc_url': 'https://cloud.google.com/access-context-manager/docs', 'name': 'Access Context Manager'}
access-transparency => {'slug': 'access-transparency', 'description': 'Audit cloud provider access', 'doc_url': 'https://cloud.google.com/cloud-provider-access-management/access-transparency/docs/overview', 'name': 'Access Transparency'}
admin-sdk => {'slug': 'admin-sdk', 'description': 'Manage Google Workspace resources', 'doc_url': 'https://developers.google.com/apps-script/advanced/admin-sdk-directory', 'name': 'Admin SDK'}
alloydb => {'slug': 'alloydb', 'description': 'Scalable & performant PostgreSQL - compatible DB', 'doc_url': 'https://cloud.google.com/alloydb/docs', 'name': 'AlloyDB'}
amp-for-email => {'slug': 'amp-for-email', 'description': 'Dynamic interactive email', 'doc_url': 'https://developers.google.com/apps-script/add-ons/gmail', 'name': 'AMP for E